In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
import re
from wordcloud import STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
import numpy as np
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

importing data

In [3]:
data=pd.read_csv("mbti_1.csv")
data_processing=data['posts']
y=data.drop('posts',axis='columns')
y.value_counts()

type
INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
dtype: int64

cleaning data 

In [7]:
def cleaning(data):
  clean_data=[]
  lm=WordNetLemmatizer()
  for i in range(len(data)):
    text=re.sub(r"[|,'@$&!*;\"]"," ",data[i])
    text=re.sub(r'http ://([^\s]+)',' ',text)
    text=re.sub(r'http://([^\s]+)',' ',text)
    text=re.sub(r'https://([^\s]+)',' ',text)
    text=re.sub(r"['-:,'_?./%?0-9(),]"," ",text)
    text=text.lower()
    text=text.split()
    for word in text:
      word=lm.lemmatize(word)
    text=list(set(text)-STOPWORDS)
    text=" ".join(text)
    clean_data.append(text)
  return clean_data
sentences=cleaning(data_processing)

Creating tokens of data ( creating matrix of words)

In [8]:
cv=CountVectorizer(max_features=5000)
x=cv.fit_transform(sentences).toarray()

encoding catagorical data 

In [9]:
le=LabelEncoder()
y=le.fit_transform(y)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Tackling imbalence in data

In [10]:
smote=SMOTE()
x_sm,y_sm=smote.fit_resample(x,y)

In [11]:
a=pd.DataFrame(y_sm)
a.value_counts()

15    1832
14    1832
13    1832
12    1832
11    1832
10    1832
9     1832
8     1832
7     1832
6     1832
5     1832
4     1832
3     1832
2     1832
1     1832
0     1832
dtype: int64

Splitting data into test and training set

In [12]:
xtrain,xtest,ytrain,ytest=train_test_split(x_sm,y_sm,test_size=0.3,random_state=0)

K-fold cross-validation with Grid search

In [14]:
parameter=[{'n_estimators':[100,150],
            'criterion':['gini'],
           'max_features':['sqrt','log2']}]
gs=GridSearchCV(estimator=rf,param_grid=parameter,scoring='accuracy',cv=5)
gs.fit(xtrain,ytrain)
best_acc=gs.best_score_
beat_param=gs.best_params_
print('Best Accuracy: {:.2f} %'.format(best_acc*100) )
print('best param:',beat_param )

Best Accuracy: 81.72 %
best param: {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 150}


Training model

In [15]:
rf=RandomForestClassifier(n_estimators=150,criterion='gini',max_features='sqrt')
rf.fit(xtrain,ytrain)


RandomForestClassifier(max_features='sqrt', n_estimators=150)

In [16]:
ypred=rf.predict(xtest)

In [17]:
print(*le.inverse_transform([ypred[0]]))

ENTJ


accuracy

In [18]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94       528
           1       0.97      0.64      0.77       544
           2       0.99      0.88      0.93       568
           3       0.86      0.68      0.76       547
           4       1.00      0.99      0.99       573
           5       1.00      0.98      0.99       545
           6       0.99      0.98      0.99       567
           7       1.00      0.96      0.98       536
           8       0.50      0.63      0.56       573
           9       0.36      0.84      0.51       556
          10       0.87      0.53      0.66       565
          11       0.56      0.62      0.59       497
          12       1.00      0.92      0.96       538
          13       1.00      0.86      0.92       557
          14       1.00      0.91      0.95       539
          15       1.00      0.83      0.90       561

    accuracy                           0.82      8794
   macro avg       0.88   